# MAS - PROJECT
## SLEA AMD - SL Evolutionary algorithm for Automatic mechanism design 
My implementation and adaptation of an evolutionary algorithm to face the problem of the Automated mechanism design. 
The goal is to define an heuristic methods to get the optimum $o_X$ (or at least something good) in an efficent way.    

In [52]:
import amd_project_functions as amd
import numpy as np
import random
import time 
import itertools 

In [53]:
# An instance
O = [[i] for i in range(20)]
Theta = [i for i in range(10)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=0, maximum_value=100)
u_instance = amd.u_class(Theta, O, minimum_value=0, maximum_value=100) 

In [54]:
# Define the fitness function V(X)
def fitness(X, Theta, p, g, u):
    return amd.v_X(X, Theta, p, g, u)

# Define how to decode a solution vector s into X
def decode_solution(s, O):
    return [O[i] for i in range(len(s)) if s[i] == 1]

# Initialize population of n individuals
def initialize_population(n, O):
    return [np.random.randint(2, size=len(O)) for _ in range(n)]

# Tournament selection
def tournament_selection(population, fitness_values, new_population_size, tournamet_size=3):
    selected = []
    for _ in range(new_population_size):
        participants = random.sample(list(zip(population, fitness_values)), tournamet_size)
        winner = max(participants, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

# Crossover
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = np.concatenate([parent1[:point], parent2[point:]])
    child2 = np.concatenate([parent2[:point], parent1[point:]])
    return child1, child2

# Mutation
def mutate(individual, mutation_rate=0.01):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]  # Flip bit
    return individual

# Main evolutionary algorithm
def evolutionary_algorithm(O, n, new_population_size, tournament_size, max_evaluations, mutation_rate, Theta, p, g, u):
    start_time = time.time()  # Record the start time
    population = initialize_population(n, O)
    evaluations = 0
    
    while evaluations < max_evaluations:
        # Fitness assessment
        fitness_values = [fitness(decode_solution(ind, O), Theta, p, g, u) for ind in population]
        evaluations += len(population)
        
        # Tournament selection
        selected_individuals = tournament_selection(population, fitness_values, new_population_size, tournament_size)
        
        # Generate new individuals through crossover and mutation
        new_population = []
        while len(new_population) < n - new_population_size:
            parents = random.sample(selected_individuals, 2)
            child1, child2 = crossover(parents[0], parents[1])
            new_population.append(mutate(child1, mutation_rate))
            if len(new_population) < n - new_population_size:
                new_population.append(mutate(child2, mutation_rate))
        
        # Create new population
        population = selected_individuals + new_population[:n - new_population_size]

    # Final fitness assessment to get the best solution
    final_fitness_values = [fitness(decode_solution(ind, O), Theta, p, g, u) for ind in population]
    best_index = np.argmax(final_fitness_values)
    best_solution = population[best_index]

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time
    
    return best_solution, round(fitness(decode_solution(best_solution, O), Theta, p, g, u),2), round(elapsed_time,2)



Find X

In [55]:
# Example usage
# An instance
O = [[i] for i in range(20)]
Theta = [i for i in range(10)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=0, maximum_value=100)
u_instance = amd.u_class(Theta, O, minimum_value=0, maximum_value=100) 

max_evaluations = 1000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
tournament_size = 3
mutation_rate = 0.05

In [56]:
best_solution, best_fitness, elapsed_time = evolutionary_algorithm(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)

print("Best solution:", best_solution)
print("Best fitness:", best_fitness)
print("Elapsed time:", elapsed_time, "seconds")

Best solution: [1 1 1 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 1]
Best fitness: 79.0
Elapsed time: 1.03 seconds


Build X

In [57]:
def decode_solution(s, O):
    """
    Given a solution vector s and a list of lists O, return the subset X
    where X contains elements from O at positions where s[i] == 1.
    
    Parameters:
    s (list of int): A binary vector indicating which elements of O are in X.
    O (list of lists): The original list of lists.
    
    Returns:
    list of lists: The subset X.
    """
    return [O[i] for i in range(len(s)) if s[i] == 1]


In [58]:
X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

Check IR for the model = True
X =  [[0], [1], [2], [3], [9], [12], [14], [15], [16], [18], [19]]
Values of oX =  79.0
         g   u
model    79.0 93.5


# Comparison with brench and bound 

## Brench and bound

In [59]:
class SearchAlgorithm:
    def __init__(self, O, Theta, p, o_XY, v_XY, g, u, L = float('-inf'), CB = set()):
        """
        Initialize the SearchAlgorithm object.
        """
        self.O = O
        self.n = len(O) - 1 # check that 
        self.Theta = Theta
        self.p = p
        self.o_XY = o_XY
        self.v_XY = v_XY
        self.g = g
        self.u = u
        self.L = L
        self.CB = CB

    def search1(self, X, Y, v, d):
        """
        Perform the SEARCH1 algorithm.
        """
        if d == self.n + 1: # end of the tree
            self.CB = X # update the current best X solution (here there is not the check that the best solution is better than the current one. No problem, it there is one before calling the function search)
            self.L = v # update the current best value of X
        else:            
            if self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search1(amd.union_listOflists_list(X,self.O[d]), Y, self.v_XY(amd.union_listOflists_list(X,self.O[d]), Y, self.O, self.Theta, self.p, self.g, self.u), d + 1)

            if self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u) > self.L:
                self.search1(X,amd.union_listOflists_list(Y,self.O[d]), self.v_XY(X, amd.union_listOflists_list(Y,self.O[d]), self.O, self.Theta, self.p, self.g, self.u), d + 1)          


def branch_and_bound_ds(O, Theta, p, o_XY, v_XY, g, u):
    start_time = time.time()  # Record the start time

    CB = None # initilize the current best solution
    L = float('-inf')
    
    SearchAlgorithm_instance = SearchAlgorithm(O, Theta, p, o_XY, v_XY, g, u, L, CB)
    SearchAlgorithm_instance.search1([], [], 0, 1)
    CB = SearchAlgorithm_instance.CB

    end_time = time.time()  # Record the end time
    elapsed_time = end_time - start_time  # Calculate the elapsed time

    return CB, round(elapsed_time,2)

## Comparison

In [60]:
# An instance
O = [[i] for i in range(20)]
Theta = [i for i in range(10)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_class(Theta, O, minimum_value=0, maximum_value=100)
u_instance = amd.u_class(Theta, O, minimum_value=0, maximum_value=100) 

max_evaluations = 3000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
tournamen_size = 3
mutation_rate = 0.05

In [61]:
def find_X(O, Theta, p, o_XY, v_XY, g_instance, u_instance):
    CB, elapsed_time = branch_and_bound_ds(O, Theta, p, o_XY, v_XY, g_instance.g, u_instance.u)
    model_vX_value = amd.v_X(CB, Theta, p, g_instance.g, u_instance.u)

    semi_random_CB = random.sample(O, len(CB))
    semi_random_vX_value = amd.v_X(semi_random_CB, Theta, p, g_instance.g, u_instance.u)
    if model_vX_value != 0:
        model_vs_rand = round((model_vX_value-semi_random_vX_value)/model_vX_value * 100,2)
    else:
        model_vs_rand = None

    return CB, round(model_vX_value,2), elapsed_time, semi_random_CB, round(semi_random_vX_value,2), model_vs_rand

CB, model_vX_value, elapsed_time, semi_random_CB, semi_random_vX_value, model_vs_rand = find_X(O, Theta, p, amd.o_XY, amd.v_XY, g_instance, u_instance)

print("Check IR for the model", amd.check_IR(Theta=Theta, X=CB, u=u_instance.u))
print("Check IR for the semi-rnd", amd.check_IR(Theta=Theta, X=semi_random_CB, u=u_instance.u))
      
print("\nElapsed time = ", elapsed_time)
print("Model X =", CB)
print("Semi random model X = ",semi_random_CB)
print("\nModel vX value ", model_vX_value)
print("Semi random model vX value ", semi_random_vX_value)
print("Relative improvement of the model ", model_vs_rand," %")

# Check the model on a single type

def check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB):
    outcomeForAllThetas_model = [amd.o_X(theta = theta, X = CB, g = g_instance.g, u = u_instance.u) for theta in Theta]
    gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
    uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

    outcomeForAllThetas_semiRandom = [amd.o_X(theta = theta, X = semi_random_CB, g = g_instance.g, u = u_instance.u) for theta in Theta]
    gForAllOutcomes_semiRandom = [g_instance.g(theta=theta, o=o_semiRandom) for theta, o_semiRandom in zip(Theta, outcomeForAllThetas_semiRandom)]
    uForAllOutcomes_semiRandom = [u_instance.u(theta=theta, o=o_semiRandom) for theta, o_semiRandom in zip(Theta, outcomeForAllThetas_semiRandom)]

    return gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom



gForAllOutcomes_model, uForAllOutcomes_model, gForAllOutcomes_semiRandom, uForAllOutcomes_semiRandom = check_model_results(CB, g_instance, u_instance, Theta, semi_random_CB)

print("        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))
print("semi rnd", round(np.mean(gForAllOutcomes_semiRandom),2), round(np.mean(uForAllOutcomes_semiRandom),2))

Check IR for the model True
Check IR for the semi-rnd True

Elapsed time =  3.97
Model X = [[6], [18], [2], [15], [5], [14], [16], [13], [3], [9]]
Semi random model X =  [[17], [10], [9], [3], [6], [19], [4], [7], [0], [14]]

Model vX value  86.7
Semi random model vX value  59.8
Relative improvement of the model  31.03  %
         g   u
model    86.7 90.5
semi rnd 59.8 90.6


In [62]:
best_solution, best_fitness, elapsed_time = evolutionary_algorithm(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

Best solution: [0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0]
Best fitness: 86.7
Elapsed time: 2.42 seconds

Check IR for the model = True
X =  [[2], [3], [6], [14], [15], [16], [18]]
Values of oX =  86.7

         g   u
model    86.7 90.5


## Bartering 

In [63]:
# An instance
number_of_items = 10
number_of_outcomes = 20
number_of_types = 5
minimum_value = 0 # try -5
maximum_value = 10 # try 5


all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value)
u_instance = amd.u_bartering_class(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value) 



In [66]:
max_evaluations = 3000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
#tournament_size = 3
#mutation_rate = 0.05 #-> the mutation rate gives problems 


best_solution, best_fitness, elapsed_time = evolutionary_algorithm(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

Best solution: [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
Best fitness: 42.2
Elapsed time: 3.98 seconds

Check IR for the model = True
X =  [(1, 0, 0, 1, 0, 0, 0, 0, 1, 0), (0, 0, 1, 0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 1, 1, 1, 1, 1, 0, 0)]
Values of oX =  42.2

         g   u
model    42.2 26.2


## Extra bartering 

In [76]:
# An instance
number_of_items = 8
number_of_outcomes = 50
number_of_types = 5
minimum_value = 0 # with [0,10] the IR constraint is mainly = False 
maximum_value = 10

all_possible_outcomes = list(itertools.product([0, 1], repeat=number_of_items))
O = random.sample(all_possible_outcomes, number_of_outcomes)
initial_goods = random.choice(O)
Theta = [i for i in range(number_of_types)] # theta in Theta is just the type of the agent (so, g will be indepedent form theta)
p = lambda theta: 1/len(Theta)
g_instance = amd.g_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods)
u_instance = amd.u_bartering_class_difference(Theta, O, minimum_value=minimum_value, maximum_value=maximum_value, initial_goods=initial_goods)  



In [94]:
max_evaluations = 2000  # Maximum number of fitness evaluations
population_size = 30 # Number of individuals in the population
new_population_size = 10  # Number of individuals to generate in each generation
#tournament_size = 3
#mutation_rate = 0.05 #-> the mutation rate gives problems 


best_solution, best_fitness, elapsed_time = evolutionary_algorithm(
    O=O,
    n=population_size,
    new_population_size=new_population_size,
    tournament_size=tournament_size,
    max_evaluations=max_evaluations,
    mutation_rate=0.1,
    Theta=Theta,
    p=p, 
    g=g_instance.g,
    u=u_instance.u
)
print("Best solution:", best_solution)
print("Best fitness:", best_fitness)
print("Elapsed time:", elapsed_time, "seconds\n")

X = decode_solution(best_solution, O)

print("Check IR for the model =", amd.check_IR(Theta=Theta, X=X, u=u_instance.u))
print("X = ", X)
print("Values of oX = ", best_fitness)

outcomeForAllThetas_model = [amd.o_X(theta = theta, X = X, g = g_instance.g, u = u_instance.u) for theta in Theta]
gForAllOutcomes_model = [g_instance.g(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]
uForAllOutcomes_model = [u_instance.u(theta=theta, o=o_model) for theta, o_model in zip(Theta, outcomeForAllThetas_model)]

print("\n        ","g  ", "u")
print("model   ", round(np.mean(gForAllOutcomes_model),2), round(np.mean(uForAllOutcomes_model),2))

Best solution: [1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 0 1 0 1 1 1 1]
Best fitness: 0.4
Elapsed time: 3.16 seconds

Check IR for the model = True
X =  [(0, 1, 0, 0, 0, 0, 1, 1), (0, 0, 0, 1, 1, 1, 0, 1), (1, 0, 1, 0, 0, 0, 1, 0), (1, 0, 0, 1, 0, 1, 0, 1), (0, 0, 1, 1, 1, 0, 0, 0), (1, 1, 0, 0, 0, 0, 1, 0), (1, 0, 1, 0, 0, 1, 0, 0), (1, 0, 1, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 1, 1, 1), (1, 0, 0, 0, 0, 1, 1, 0), (0, 1, 1, 0, 1, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 1, 1, 0, 1, 0), (0, 1, 0, 0, 1, 1, 0, 0), (0, 0, 1, 1, 0, 1, 0, 1), (0, 1, 0, 1, 1, 1, 0, 0), (0, 1, 1, 0, 0, 0, 1, 0)]
Values of oX =  0.4

         g   u
model    0.4 7.6
